# Multiprocessng Federated MNIST torch Tutorial

## !!WARNING!! 
Multiprocessing is not stable in Jupyter Notebook. Some processes can live in the memory when the main already finished.
We recomended to restart the kernel after every experiment.

To run federation in multiprocessing way you need: 
1. define model class `Net`, and functions thats return opimizer `get_optimizer` and loss function `cross_entropy` in **in separate cell**. 
2. save jupyter notebook cell as file using `%%writefile <filename>` in the top of the cell.
2. import `Net`, `cross_entropy` and `get_optimizer` in the next cell
   from <filename> import Net, cross_entropy, get_optimizer

In [4]:
% % writefile torch_model.py

import torch.nn as nn


class Net(nn.Module):
    """PyTorch Neural Network."""

    def __init__(self):
        """Initialize."""
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 * 5 * 5, 32)
        self.fc2 = nn.Linear(32, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        """Forward pass of the network."""
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def cross_entropy(output, target):
    """Binary cross-entropy metric."""
    return F.cross_entropy(input=output, target=target)


def get_optimizer(x):
    """Optimizer function."""
    return optim.Adam(x, lr=1e-4)


Overwriting torch_model.py


In [2]:
!pip install openfl torchvision

In [1]:
# Workaround to run multiprocessing scenario in Jupyter Notebook
# Don't define those instances in this cell directily
from torch_model import Net, cross_entropy, get_optimizer

import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import openfl.native as fx
from openfl.federated import FederatedModel, FederatedDataSet
from openfl.interface.cli import setup_logging


def one_hot(labels, classes):
    """One-hot encode `labels` using `classes` classes."""
    return np.eye(classes)[labels]


setup_logging()

t = time.time()

fx.init('torch_cnn_mnist')

is_multi = True
batch_size = 32
rounds_to_train = 2
collaborators_amount = 5
classes = 10

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5), (0.5))])

trainset = datasets.MNIST(root='./data',
                          train=True,
                          download=True,
                          transform=transform)

validset = datasets.MNIST(root='./data',
                          train=False,
                          download=True,
                          transform=transform,
                          target_transform=lambda labels: one_hot(labels, classes))

(train_images, train_labels), (valid_images, valid_labels) = (zip(*dataset) for dataset in
                                                              [trainset, validset])
train_images, valid_images = (torch.stack(images).numpy() for images in
                              [train_images, valid_images])
train_labels, valid_labels = (np.stack(labels) for labels in [train_labels, valid_labels])
feature_shape = train_images.shape[1]

fl_data = FederatedDataSet(train_images, train_labels, valid_images, valid_labels,
                           batch_size=batch_size, num_classes=classes)
fl_model = FederatedModel(build_model=Net, optimizer=get_optimizer,
                          loss_fn=cross_entropy, data_loader=fl_data)
collaborator_models = fl_model.setup(num_collaborators=collaborators_amount)
collaborators = {str(i): c for i, c in enumerate(collaborator_models)}

print(f'Original training data size: {len(train_images)}')
print(f'Original validation data size: {len(valid_images)}\n')

final_fl_model = fx.run_experiment(collaborators, {
    'aggregator.settings.rounds_to_train': rounds_to_train,
}, is_multi=is_multi)
final_fl_model.save_native('final_pytorch_model')
print(f'FINISH in {(time.time() - t):.2f}')


Creating Workspace Directories
Creating Workspace Templates

New workspace directory structure:
workspace
├── torch_model.py
├── requirements.txt
├── final_pytorch_model
├── plan
│   ├── defaults
│   │   ├── tasks_keras.yaml
│   │   ├── data_loader.yaml
│   │   ├── tasks_torch.yaml
│   │   ├── tasks_tensorflow.yaml
│   │   ├── network.yaml
│   │   ├── assigner.yaml
│   │   ├── defaults
│   │   ├── compression_pipeline.yaml
│   │   ├── tasks_fast_estimator.yaml
│   │   ├── aggregator.yaml
│   │   ├── collaborator.yaml
│   │   └── task_runner.yaml
│   ├── cols.yaml
│   ├── data.yaml
│   └── plan.yaml
├── save
│   ├── torch_cnn_mnist_init.pbuf
│   ├── torch_cnn_mnist_best.pbuf
│   ├── keras_cnn_mnist_init.pbuf
│   ├── keras_cnn_mnist_last.pbuf
│   ├── keras_cnn_mnist_best.pbuf
│   └── torch_cnn_mnist_last.pbuf
├── .workspace
├── agg_to_col_two_signed_cert.zip
├── data
│   ├── MNIST
│   │   ├── raw
│   │   └── processed
│   ├── cifar-10-batches-py
│   │   ├── batches.meta
│   │   ├── data_

FINISH in 28.08


## !!NOTICE!!  Don't forget to reload kernel after execution
<img align="left" src="reload_kernel.png">